# Big Data - Aula 5 - parte 2 
---



# Algoritmic Concepts and Distributed Computing Challenges


---


# 1) Graph Based Data - Inverted Indexing
----

An inverted index is a data structure that makes it easy, given a term, to find (pointers to) all the places where that term occurs.


<img src="images/inv_index.png" style="width:70%" />


### Vamos implementar um inverted index?

Considerem o seguinte texto, dividido por pontos finais:

In [6]:
raw = "In statistics, maximum likelihood estimation(MLE) is a method of estimating the parameters of a statistical model given observations, by finding the parameter values that maximize the likelihood of making the observations given the parameters. MLE can be seen as a special case of the maximum a posteriori estimation(MAP) that assumes a uniform prior distribution of the parameters, or as a variant of the MAP that ignores the prior and which therefore is unregularized. The method of maximum likelihood corresponds to many well-known estimation methods in statistics. For example, one may be interested in the heights of adult female penguins, but is unable to measure the height of every single penguin in a population due to cost or time constraints. Assuming that the heights are normally distributed with some unknown mean and variance, the mean and variance can be estimated with MLE while only knowing the heights of some sample of the overall population. MLE would accomplish this by taking the mean and variance as parameters and finding particular parametric values that make the observed results the most probable given the model. In general, for a fixed set of data and underlying statistical model, the method of maximum likelihood selects the set of values of the model parameters that maximizes the likelihood function. Intuitively, this maximizes the 'agreement' of the selected model with the observed data, and for discrete random variables it indeed maximizes the probability of the observed data under the resulting distribution. Maximum likelihood estimation gives a unified approach to estimation, which is well-defined in the case of the normal distribution and many other problems."

raw_list = raw.split(".")
len(raw_list)

10

In [9]:
"{}.".format(raw_list[0])

'In statistics, maximum likelihood estimation(MLE) is a method of estimating the parameters of a statistical model given observations, by finding the parameter values that maximize the likelihood of making the observations given the parameters.'

In [10]:
"{}.".format(raw_list[1])

' MLE can be seen as a special case of the maximum a posteriori estimation(MAP) that assumes a uniform prior distribution of the parameters, or as a variant of the MAP that ignores the prior and which therefore is unregularized.'

In [16]:
i_index = {}
i_index["in"] = {}
i_index["likelihood"] = {}
i_index["in"][0] = True #word in, appears in doc 0, 1 time
i_index["likelihood"][1] = True #word in, appears in doc 1, 1 time
i_index["in"][1] = True

print(i_index)

j_index = {}
j_index["in"] = []
j_index["in"].append(0)
j_index["in"].append(1)
i_index["likelihood"][1] = True
j_index


{'in': {0: True, 1: True}, 'likelihood': {1: True}}


{'in': [0, 1]}

## Trabalho de Casa 1

1) Implement in python a function i_index_gen() that given as input a set of documents (an array of strings) returns an  inverted index based. Clean the text from punctuations chars or upper letters. 

2) Make a second function that returns the same i_index based on a pandas DataFrame without indexes

3) Make a third function that returns the same i_index based on a pandas DataFrame using indexes

4) Compare the performance for the three methods using ```%%time``` (you should increase your text example) and take your conclusions how lists, DataFrames (with or without indexes) manage searches


**Don't forget to**
- present all results in a python notebook;
- Comment your code;

Good Luck

In [4]:
# Example on the usage of %timeit
import math
%timeit [math.cos((i/2**i)**3) for i in range(1,10000)]
print("----")
%timeit [(i)**3 for i in range(1,10000)]

91.4 ms ± 3.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
----
3.88 ms ± 463 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%%time
print("ola")

ola
CPU times: user 216 µs, sys: 97 µs, total: 313 µs
Wall time: 264 µs


---

#### Problems in Big Data Scenario?

---

- Inverted indexing is fundamentally a very large search problem.
- Again, problems arise when data does not fit in memory or a single CPU is too short.
- How to deal with millions of files or a giant ```i_index```? 

# 2) Graph Based Algorithms

##  Page Rank Applications

#### Basic Idea: Links as votes for Ranking pages

Each link’s vote is proportional to the importance of its source page

- If page $j$ with importance $r_j$ has $n$ out-links, each link gets $r_j / n_j$ votes
- Page $j$’s own importance is the sum of the votes on its in-links (I) (A page is important if it is pointed to by other important pages)

So, for a single page $j$, its importance $r_j$ is given by:

$$r_j = \sum_{i=1}^{I} \frac{r_i}{n_i}$$

---

Take a first set of 3 nodes:

<img src="images/page_rank.png" style="width:30%"/>





## Let's write the equations, starting in $y$:

$$r_y =r_y/2+r_a/2 $$

$$r_a =r_y/2+r_m$$ 

$$r_m =r_a /2$$

Is this solvable? What is wrong? The underlying matrix is $Singular$. 
We need a fourth equation to give this a solution:

$$r_m + r_a + r_y = 1$$
---


Let's build a matrix M where this equations are represented in the following order (both rows and columns) $[y, a, m]$
Our system (Homogenous) is then 


$$ 
\begin{bmatrix} 
1/2 & -1/2 & 0\\
-1/2 & 1 & -1\\
0 & -1/2 & 1\\
\end{bmatrix}
\quad
\begin{bmatrix} 
y\\
a\\
m\\
\end{bmatrix} = 
\begin{bmatrix} 
0\\
0\\
0\\
\end{bmatrix}$$

In [244]:
import numpy as np

M = np.array([[0.5, -0.5, 0],
[-0.5, 1, -1],
[0, -0.5, 1]])
print(M)

np.linalg.det(M)

[[ 0.5 -0.5  0. ]
 [-0.5  1.  -1. ]
 [ 0.  -0.5  1. ]]


0.0

We have to add an extra relation between parameters: for instance, a normalization information their relative values like $ y+a+m = 1$, which means a = 1-y-m. Let's use this in the last equation, obtaining $(y+m-1)/2 + m = 0 <=> y/2 + 3/2m  = 1/2$.



$$ 
\begin{bmatrix} 
1/2 & -1/2 & 0\\
-1/2 & 1 & -1\\
1/2 & 0 & 3/2\\
\end{bmatrix}
\quad
\begin{bmatrix} 
y\\
a\\
m\\
\end{bmatrix} = 
\begin{bmatrix} 
0\\
0\\
1/2\\
\end{bmatrix}$$

In [245]:
M = np.array([[0.5, -0.5, 0],
[-0.5, 1, -1],
[1/2, 0, 3/2]])

#np.linalg.inv(M)
b= np.array([[0], [0], [1/2]])

X = np.linalg.solve(M,b)
X

array([[ 0.4],
       [ 0.4],
       [ 0.2]])

But now if M is huge?
linalg uses gaussian elimination. this is not good for huge matrices (Big Data).

Inverting huge matrices is a very relvant subject in general. Also here.

## BIGDATA ALGORITHM: POWER ITERATION

## Matrix vector Multiplication (e.g. Page Rank)

  

As we saw before, Gaussian elimination method works for small examples, but we need a better method for large web-size graphs


So, for a single page $j$, its importance $r_j$ is given by:

$$r_j = \sum_{i=1}^{I} \frac{r_i}{n_i}$$

---

Take a first set of 3 nodes:

<img src="images/page_rank.png" style="width:30%"/>


Could be written as 

$$r_y =r_y/2+r_a/2 $$

$$r_a =r_y/2+r_m$$ 

$$r_m =r_a /2$$


We have to add an extra relation between parameters: for instance, a normalization information their relative values like $ y+a+m = 1$, which means a = 1-y-m. 



$$ 
\begin{bmatrix} 
1/2 & -1/2 & 0\\
-1/2 & 1 & -1\\
1/2 & 0 & 3/2\\
\end{bmatrix}
\quad
\begin{bmatrix} 
y\\
a\\
m\\
\end{bmatrix} = 
\begin{bmatrix} 
0\\
0\\
1/2\\
\end{bmatrix}$$



### What about if this matrix is HUGE?

To solve this problem let's move back again to the original formulation and consider the matrix $M$ with dimensions D by D where D = number of web pages and  where $M_{ij}$ is given by 

``` If i == j then ```  

$ \space\space\space M_{ij} = 1/d_i$

``` else```

$ \space\space\space M_{ij} =  0$

where $d_i$ are the number of outbound links;

This is exactly the right side of 


$$r_y =r_y/2+r_a/2 $$

$$r_a =r_y/2+r_m$$ 

$$r_m =r_a /2$$

This relations are called the **flow equations** and they could be written as 

$$r = Mr$$

where $r = [r_a r_y r_m]^T$ which are the page ratings.

Suppose page $i$ links to 3 pages, including $j$, we the flow equation states that $r_j = \sum_{i=1}^{I} \frac{r_i}{n_i}$.


<img src="images/flow_equation.png" style="width:60%"/>


If $Mr = r$ this means $Mr = Ir = \lambda r$

This means $r$ is a so called eigenvector of $M$ 
Actually, since $||\sum_i(M_{ij})|| < 1$ (due to normalization) and since $\lambda=1$, then $r$ is the biggest eigenvalue of $M$.

> $x$ is an eigenvector of $A$ if $Ax = \lambda x$ where $\lambda$ is a real constant 

----

## Power iteration: 
### A simple iterative scheme to find the dominant eigenvector (r)

- Suppose there are N web pages
- Initialize: $r^{(0)} = [1/N,....,1/N]^T$

> This means all rankings are considered equal at the begining.

- Iterate: $r^{(t+1)} = M.r^{(t)}$
  
- Stop when $\|r(t+1) – r(t)\| < ε$

----


## How it Works?

$r^{(2)} = Mr^{(1)} = M(Mr^{(0)}) = M^2r^{(1)}$

Claim: $M^k r^{(1)} \rightarrow r$


### Proof:
$r$ could be written using a linear combination of the M eigenvectors x_1,...x_n (all). That means:

$$r^{(0)} = c_1 x_1 + c_2 x_2 + ... + c_n x_n$$

then

$$Mr^{(0)} = M(c_1 x_1 + c_2 x_2 + ... + c_n x_n) = c_1 \lambda_1 x_1 + c_2 \lambda_2 x_2 + ... + c_n \lambda_n x_n$$

finally

$$M^k r^{(0)} = c_1(\lambda_1^k x_1) + c_2(\lambda_2^k x_2) + ... + c_n(\lambda_n^k x_n)$$

since all $\lambda$s (eigenvalues) are $<1$ besides $\lambda_1=1$ (the one we want), when $k -> \inf$

$$M^k r^{(0)} -> c_1(x_1)$$

But since $Mr = r$ for the right $r$, then $c_1 = 1 $

---

### The Random Web Surfer Interpretation


Imagine a random web surfer

- At any time $n$ surfer is on some page $i$ 
- At time $n+1$, the surfer follows an out-link from $i$ uniformly at random 
- Ends up on some page $j$ linked from $i$
- Process repeats indefinitely


$r_j = \sum_{i=1}^{N} \frac{r_i}{n_i}$

Consider $p(n)$ as the probability distribution of the websurfer at time $n$

**What is the probability of the web surfer to be in the page $j$ at time $n+1$?**

<img src="images/page_rank_as_prob.png" style="width:40%"/>

Since we are navigating from anywhere, it depends on the probabilities of being in $i_1$, $i_2$ or $i_3$ at the previous step (n), and then, at random ($1/d_i$) jump to this page. That is,

$$p(n+1) = M p(n)$$

Suppose now that we reach 

$$p(n+1) = M p(n) = p(n)$$

Which means, the probability distribution is not altered anymore by new steps. Then $p(t)$ is said to be a **stationary distribution of a random walk**.

### Some Remarks

- If we solve $p(n)$, random web surfer probability map, we can find or original $r$ vector (rank vector for each page). 
- So, each time we multiply $M$ by $r^k$ ($Mr^k$) we are calculating a new $p(n+1)$. 
- Each row in M takes the role of probabiltiy distribution to go from previous positions to new positions at n+1 time.

** This was Google problem to be solved back in 1998. ***

Let us finally see some issues to define the final equation to be solved.

---

## Issues to solve the Final Problem

**2 problems when solving the random walking problem**

(1) Some pages are **dead ends** (have no out-links)
>    Random walk has “nowhere” to go to
>  - this leaks the resulting ranks

(2) **Spider traps** (all out-links are within the group)
>  Random walked gets “stuck” in a trap
>  - And eventually spider traps absorb all importance
  
<img src="images/page_rank_probs.png" style="width:30%"/>


**Google Solved this With Random TelePorts - Some times we should just randomly jump out to any other page to avoid dead ends and spider traps**


Consider this random jump is done with probability $1-\beta$ and not done with probability $\beta$. 
This means each line of M has not zeros it allways have some probability of jumping to an unexpected page!

The new flow equation is now:

$$r_j = \beta \sum_i \frac{r_i}{d_i} + (1-\beta)\frac{1}{N}$$



And the new Google Matrix A is:

$$A = \beta M + (1-\beta) \left[\frac{1}{N} \right]_{N \times N}$$
$$ $$
<div style="text-align:center">[Brin-Page, ‘98]</div>

http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf

## Finally, applying to Page Rank Calculation

Key step is the following matrix-vector multiplication:

$$r_{new} =A r_{old}$$

This is easy if we have enough main memory to hold $A, r_{old}, r_{new}$

Say N = 1 billion pages   
We need 4 bytes for each entry (typical...)

- aprox 8GB for each vector, that's high but solvable..

- BUT Matrix A has N^2 entries where $N = 10^9$ -->   $10^{18}$ is a large number (plus 4 Bytes per entry!)

This is infeasible. The final page rank equation should be: 

$$r_{new} = \beta M r_{old} + \left[\frac{1-\beta}{N} \right]_{N \times 1}$$


M is sparse, so everything is feasible.

## 2) Matrix Multiplication

Matrix multiplication is used for almost any machine learning model creation since most methods are nowadays stronlgy supported by linear algebra.

For instance: 

- Distance measure;
- Gradient Calculation;
- Graph Calculation;

Let's look an example in distance measure between x points.

In [ ]:
import numpy as np
A = np.round(np.random.random([4,4])*100)
A = np.tril(A)

print(A)
b = np.ones([4,1])*2
print(b)

np.matmul(A,b)

---

#### Problems in Big Data Scenario?

---

- If Initial Matrix too large for memory? Apply a row multiplication per node!?

- If not even a single line is able to be fitted in memory? Notice that each input in the resulting vector depends entirely on a single line of the matrix

<img src="./images/matmul.png"/>

## How do this applies to distance calculation?

How to calculate the distance between this two points (a,b) in $R^N$?


$$D(\vec{a},\vec{b}) = \sqrt{(a_x^2 - b_x^2) + (a_y^2 - b_y^2) + ...} = \sqrt{\sum_i^N{\Delta ab_i ^2}} $$

This is the so called Norm2 distance. There are others like Norm1 (Manhathan distance) where $D(a,b) = \sum_i^{N}{|\Delta ab_i|}$

Consider a matrix 


$$W =  \begin{bmatrix}
    2 & -1 \\
    5 & 1 \\
    6 & 9 \\
    -2 & 3    
\end{bmatrix} = \begin{bmatrix}
    a \\
    b \\
    c \\
    d \\    
\end{bmatrix}$$

Let's compute the norm2 between each pair of points using normal programming techniques:

In [194]:
import numpy as np, math
W = [[2, 1], [5,1], [6,9], [-2,3]]
W = np.array(W)
W

array([[ 2,  1],
       [ 5,  1],
       [ 6,  9],
       [-2,  3]])

In [196]:
NW = []
for w1 in W:
    NW.append([])
    for w2 in W:
        NW[-1].append(0)
        for d in range(len(W[0])):
            NW[-1][-1] += (w1[d]-w2[d])**2
        NW[-1][-1] = math.sqrt(NW[-1][-1])
        
M = np.array(NW)

In [197]:
print(M)
print("----------------")
print(M-M.T)

[[  0.           3.           8.94427191   4.47213595]
 [  3.           0.           8.06225775   7.28010989]
 [  8.94427191   8.06225775   0.          10.        ]
 [  4.47213595   7.28010989  10.           0.        ]]
----------------
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [208]:
M = np.zeros((W.shape[0],W.shape[0]))

for d in range(W.shape[1]):
    w = np.repeat(W[:,d].reshape(-1,1),W.shape[0], axis=1)
    M += (w-w.T)**2
np.sqrt(M)

array([[  0.        ,   3.        ,   8.94427191,   4.47213595],
       [  3.        ,   0.        ,   8.06225775,   7.28010989],
       [  8.94427191,   8.06225775,   0.        ,  10.        ],
       [  4.47213595,   7.28010989,  10.        ,   0.        ]])

In [214]:
W = np.random.random([10000,2])
W

array([[ 0.69168128,  0.95795148],
       [ 0.69728775,  0.58615352],
       [ 0.03548373,  0.96888841],
       ..., 
       [ 0.27724978,  0.92632415],
       [ 0.23730736,  0.81649573],
       [ 0.57104187,  0.10000069]])

In [212]:
%%time
NW = []
for w1 in W:
    NW.append([])
    for w2 in W:
        NW[-1].append(0)
        for d in range(len(W[0])):
            NW[-1][-1] += (w1[d]-w2[d])**2
        NW[-1][-1] = math.sqrt(NW[-1][-1])
        
M = np.array(NW)
print(M)

[[ 0.          0.58875073  0.78891475 ...,  0.17017593  0.0838973
   0.89904977]
 [ 0.58875073  0.          0.65226429 ...,  0.50255283  0.50536241
   0.74941776]
 [ 0.78891475  0.65226429  0.         ...,  0.61887957  0.73582055
   0.11302332]
 ..., 
 [ 0.17017593  0.50255283  0.61887957 ...,  0.          0.12752133
   0.72933304]
 [ 0.0838973   0.50536241  0.73582055 ...,  0.12752133  0.          0.84764047]
 [ 0.89904977  0.74941776  0.11302332 ...,  0.72933304  0.84764047  0.        ]]
CPU times: user 2.92 s, sys: 9.23 ms, total: 2.93 s
Wall time: 2.93 s


In [215]:
%%time
m = np.zeros((W.shape[0],W.shape[0]))

for d in range(W.shape[1]):
    w = np.repeat(W[:,d].reshape(-1,1),W.shape[0], axis=1)
    m += (w-w.T)**2
    
M = np.sqrt(m)
print(M)

[[ 0.          0.37184023  0.65628869 ...,  0.41563657  0.47588379
   0.86639103]
 [ 0.37184023  0.          0.76450674 ...,  0.54050713  0.51443123
   0.5022774 ]
 [ 0.65628869  0.76450674  0.         ...,  0.2454843   0.25289584
   1.02068035]
 ..., 
 [ 0.41563657  0.54050713  0.2454843  ...,  0.          0.11686607
   0.87699729]
 [ 0.47588379  0.51443123  0.25289584 ...,  0.11686607  0.          0.7904074 ]
 [ 0.86639103  0.5022774   1.02068035 ...,  0.87699729  0.7904074   0.        ]]
CPU times: user 5.95 s, sys: 2.44 s, total: 8.38 s
Wall time: 8.67 s


In [ ]:
%%time
#W = np.random.random([1000,2])
w = (np.repeat(W.reshape(-1,1,W.shape[1]),W.shape[0], axis=1)-np.repeat(W.reshape(1,-1,W.shape[1]),W.shape[0], axis=0))**2
M = np.sum(w, axis=2)
print(M)